<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/GloVe_XGB_gossipcop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RQ1: Glove embeddings for feature Title and Text of the news article. Embeddings fed into the XgBoost ML model to see results. 


In [1]:
!pip install -U xgboost wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 192.9 MB 56 kB/s 
     |████████████████████████████████| 1.8 MB 53.6 MB/s 
     |████████████████████████████████| 146 kB 61.2 MB/s 
     |████████████████████████████████| 181 kB 55.4 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=f411d31ade6538419c04c021a9477ff6201c7ff8cf6f66a73c566bef942c9a42
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import gensim.downloader as api

import xgboost as xgb

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from pathlib import Path

import wandb
from wandb.xgboost import WandbCallback
from IPython.display import clear_output


metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return 'eval/' + score_func.__name__.split("_")[0]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = Path("/content/drive/MyDrive/ResearchFND")
assert base_dir.exists()

Mounted at /content/drive


In [4]:
dataset_id = 'gossipcop'

In [8]:
df = pd.read_csv(base_dir/f"{dataset_id}_agg.csv")
df.text.fillna('', inplace=True)
df.head()

,title,text,tweets,retweets,label,url
0,Kendall Kylie Jenner Jenner NOT Upset Up...,,[],"['995423424741888001', '995461685166202880', '...",fake,NaN
1,Kim Kardashian Dethroned Dethroned By Khlo...,,[],"['848843565027516416', '849030801970868224', '...",fake,NaN
2,Kim Kardashian Did NOT Hot Staffer Hot Sta...,,[],"['940685393112064001', '977921622672920576', '...",fake,NaN
3,The Voice The Voice Team NOT Surprised Sur...,,[],[],fake,NaN
4,Drake NOT Angelina Jolie s Toy Boy Toy Boy ...,,"[{'id': 948630026496323585, 'text': 'Drake NOT...","['948022124626808832', '948630026496323585', '...",fake,NaN


In [9]:
(df.title.str.len() == 0).any()

False

In [10]:
assert len(df) == len(df.title.unique())

In [11]:
news = df[['title', 'text', 'label']].copy()

print(news.isna().any())
news.count()

title    False
text     False
label    False
dtype: bool


title    19968
text     19968
label    19968
dtype: int64

Importing libraries to preprocess the title column. 

In [12]:
import nltk
nltk.download('stopwords')

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
news['text']= news['text'].apply(denoise_text)

In [14]:
#Tokenizing Text -> Repsesenting each word by a number
tokenizer = text.Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(news.title)

In [15]:
d_wv = 300
glove = api.load(f'glove-wiki-gigaword-{d_wv}')

[==================================================] 100.0% 376.1/376.1MB downloaded


Creating the embedding matrix to convert features to model-readable format. 

In [16]:
emb_matrix = np.zeros((max(tokenizer.word_index.values())+1, d_wv), dtype=np.float32)
for i in range(1,max(tokenizer.word_index.values())+1):
    word = tokenizer.index_word[i]
    if word in glove.vocab:
        emb_matrix[i, :] = glove.word_vec(word)

In [17]:
import ast

df['tweets'] = df.tweets.map(ast.literal_eval)

Prepping the features.

In [18]:
df["num_retweets"] = df.retweets.map(lambda x: len(x.split(',')))
df["log_num_retweets"] = np.log1p(df.num_retweets.to_numpy())
df["num_tweets"] = df.tweets.map(lambda x: len(set([t["id"] for t in x])))
df["log_num_tweets"] = np.log1p(df.num_tweets.to_numpy())

# Training

In [19]:
WANDB_ENTITY = 'saloniteam'
WANDB_PROJECT = 'nofolds'

The wandb is already logged in a session. Access key to be provided if login is not done. It logs the results into the *nofolds* project in the saloniteam entity (name of account) 

Training the model for titles and full text below respectively. Taking the tokenised text, defining the train parameters, specifiying the name of the run which will be logged in wandb. Once the run is complete, a short summary and the link to wandb is displayed. The wandb tracking can be customised to include different evaluation metrics, and run statistics as well. 

## Titles

In [20]:
tokenized_titles = tokenizer.texts_to_sequences(news.title.tolist())

X = []
for title in tokenized_titles:
    X.append(emb_matrix[title].mean(0))

X = np.stack(X, axis=0)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((19968, 300), (19968,))

In [21]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [22]:
def train(train_idx, test_idx, params):

    # training
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain,'train'), (dtest,'eval')]
    N = params.get('num_boost_round', 200)
    clf = xgb.train(params, dtrain, num_boost_round=N, early_stopping_rounds=None, evals=watchlist, callbacks=[WandbCallback()])
    #evaluation
    probs = clf.predict(dtest)
    y_pred = (probs > 0.5).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})

In [23]:
params = {
    "objective":'binary:logistic',
    "seed":124,
    'num_boost_round':200
}

In [24]:
GROUP = f"{dataset_id}-me-glove-title-xgb"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity=WANDB_ENTITY, project=WANDB_PROJECT, group=GROUP, name=f"{GROUP}-{fold_id}") as run:
        train(train_idx, test_idx, params)
    break

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


[19:18:43] WARNING: ../src/learner.cc:627: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-logloss:0.58670	eval-logloss:0.60014
[1]	train-logloss:0.52188	eval-logloss:0.54931
[2]	train-logloss:0.47744	eval-logloss:0.51655
[3]	train-logloss:0.44686	eval-logloss:0.49819
[4]	train-logloss:0.42172	eval-logloss:0.48486
[5]	train-logloss:0.40078	eval-logloss:0.47534
[6]	train-logloss:0.38394	eval-logloss:0.46910
[7]	train-logloss:0.36684	eval-logloss:0.46478
[8]	train-logloss:0.35352	eval-logloss:0.45999
[9]	train-logloss:0.34141	eval-logloss:0.45613
[10]	train-logloss:0.33027	eval-logloss:0.45353
[11]	train-logloss:0.31975	eval-logloss:0.45098
[12]	train-logloss:0.31059	eval-logloss:0.45022
[13]	train-logloss:0

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▃▂▁▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇██
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
train-logloss,█▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,199
eval/accuracy,0.81948
eval/f1,0.89051
eval/precision,0.83795


## Fulltext

Training for full text + titles now. It can be compared to the performance above to see if there was an improvement or not by adding another feature for real/fake news prediction. 

In [25]:
news['fulltext'] = news['title'] + '\n' + news['text']

In [26]:
#Tokenizing Text -> Repsesenting each word by a number
tokenizer = text.Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(news.fulltext)

In [27]:
emb_matrix = np.zeros((len(tokenizer.word_index)+1, d_wv), dtype=np.float32)
for i in range(1,max(tokenizer.word_index.values())+1):
    word = tokenizer.index_word[i]
    if word in glove.vocab:
        emb_matrix[i, :] = glove.word_vec(word)
print(emb_matrix.shape)

(155471, 300)


In [28]:
tokenized_texts = tokenizer.texts_to_sequences(news.fulltext.tolist())

# maximal number of tokens to use (set to -1 to use full article)
max_len = 128 
X = []
for text_ids in tokenized_texts:
    text_ids = text_ids[:max_len]
    X.append(emb_matrix[text_ids].mean(0))

X = np.stack(X, axis=0)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((19968, 300), (19968,))

In [29]:
params = {
    "objective":'binary:logistic',
    "seed":124,
    'num_boost_round':200
}

In [30]:
GROUP = f"{dataset_id}-me-glove-fulltext-xgb"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity=WANDB_ENTITY, project=WANDB_PROJECT, group=GROUP, name=f"{GROUP}-{fold_id}", tags=['glove', 'xgb']) as run:
        train(train_idx, test_idx, params)
    break

wandb: Currently logged in as: saloni (saloniteam). Use `wandb login --relogin` to force relogin


[19:21:55] WARNING: ../src/learner.cc:627: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-logloss:0.58040	eval-logloss:0.59681
[1]	train-logloss:0.51274	eval-logloss:0.53984
[2]	train-logloss:0.46633	eval-logloss:0.50653
[3]	train-logloss:0.43331	eval-logloss:0.48344
[4]	train-logloss:0.40866	eval-logloss:0.47011
[5]	train-logloss:0.38701	eval-logloss:0.45858
[6]	train-logloss:0.37183	eval-logloss:0.45281
[7]	train-logloss:0.35518	eval-logloss:0.44776
[8]	train-logloss:0.34178	eval-logloss:0.44280
[9]	train-logloss:0.33063	eval-logloss:0.44081
[10]	train-logloss:0.31863	eval-logloss:0.43782
[11]	train-logloss:0.30761	eval-logloss:0.43418
[12]	train-logloss:0.29898	eval-logloss:0.43039
[13]	train-logloss:0

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▃▂▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
train-logloss,█▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,199
eval/accuracy,0.83425
eval/f1,0.8989
eval/precision,0.85009


## + num tweets/retweets

In [31]:
glove_text_embs = X.copy()

In [32]:
X = np.concatenate([glove_text_embs, df.num_retweets.to_numpy()[..., None], df.num_tweets.to_numpy()[..., None]], axis=1)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((19968, 302), (19968,))

In [33]:
GROUP = f"{dataset_id}-me-glove-fulltext+tw-xgb"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity=WANDB_ENTITY, project=WANDB_PROJECT, group=GROUP, name=f"{GROUP}-{fold_id}", tags=['glove', 'xgb']) as run:
        train(train_idx, test_idx, params)
    break

[19:24:50] WARNING: ../src/learner.cc:627: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-logloss:0.54947	eval-logloss:0.56159
[1]	train-logloss:0.46563	eval-logloss:0.48752
[2]	train-logloss:0.41149	eval-logloss:0.44420
[3]	train-logloss:0.37356	eval-logloss:0.41588
[4]	train-logloss:0.34520	eval-logloss:0.39685
[5]	train-logloss:0.32312	eval-logloss:0.38530
[6]	train-logloss:0.30408	eval-logloss:0.37242
[7]	train-logloss:0.28808	eval-logloss:0.36498
[8]	train-logloss:0.27410	eval-logloss:0.35810
[9]	train-logloss:0.26130	eval-logloss:0.35396
[10]	train-logloss:0.24960	eval-logloss:0.35002
[11]	train-logloss:0.24017	eval-logloss:0.34684
[12]	train-logloss:0.23036	eval-logloss:0.34404
[13]	train-logloss:0

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
train-logloss,█▆▅▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,199
eval/accuracy,0.87732
eval/f1,0.92325
eval/precision,0.89357


The run can be found in the specified group by name or direct link to wandb. 